In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from imblearn.over_sampling import SMOTE

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
from sklearn import metrics
import matplotlib.pyplot as plt

**TAREFA 1**

In [ ]:
#linfócito B
bcell = pd.read_csv('../input/epitope-prediction/input_bcell.csv')
bcell

In [ ]:
protseq = []

for i in bcell.protein_seq:
    tam = len(i)
    protseq.append(tam)

pepseq = bcell.end_position - bcell.start_position

In [ ]:
protseq = pd.DataFrame(protseq)
bcell['protein_seq']= (bcell['protein_seq']==0).astype(int)
bcell = bcell.replace({'protein_seq': protseq})

pepseq = pd.DataFrame(pepseq)
bcell['peptide_seq'] = (bcell['peptide_seq']==0).astype(int)
bcell = bcell.replace({'peptide_seq': pepseq})
bcell

In [ ]:
#atribitos numéricos
atributos = ['protein_seq', 'peptide_seq', 'chou_fasman', 'kolaskar_tongaonkar', 'parker', 'isoelectric_point', 'aromaticity', 'hydrophobicity', 'stability']
celb = bcell[atributos]
celb.isnull().sum()

In [ ]:
celb

In [ ]:
bcell.groupby('target').hist(figsize=(10, 10), color = 'gray')

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(celb,bcell['target'],test_size=0.33, random_state=1)

print("Antes do SMOTE, número de target '1': {}".format(sum(bcell.target==1)))
print("Antes do SMOTE, número de target '0': {} \n".format(sum(bcell.target==0)))

bcell['target'].hist(figsize=(20, 10), color = 'purple')

sm = SMOTE(random_state=2)
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)

print("Depois do SMOTE, número de target '1': {}".format(sum(y_train_sm==1)))
print("Depois do SMOTE, número de target '0': {} \n".format(sum(y_train_sm==0)))

In [ ]:
print(f'''X antes do SMOTE: {X_train.shape}
X depois do SMOTE: {X_train_sm.shape}''')

In [ ]:
#comparação dos modelos

dt = DecisionTreeClassifier(max_leaf_nodes=30, random_state=1)
dt = dt.fit(X_train_sm, y_train_sm)
y_pred_dt = dt.predict(X_test)

rf = RandomForestClassifier(n_estimators=30, random_state=1)
rf = rf.fit(X_train_sm, y_train_sm)
y_pred_rf = rf.predict(X_test)


print("ema dt:", metrics.mean_absolute_error(y_test, y_pred_dt))
print("ema rf:", metrics.mean_absolute_error(y_test, y_pred_rf))
print("acc dt:", metrics.accuracy_score(y_test, y_pred_dt))
print("acc rf:", metrics.accuracy_score(y_test, y_pred_rf))

In [ ]:
print('recall dt:', metrics.recall_score(y_test, y_pred_dt))
print('recall rf:', metrics.recall_score(y_test, y_pred_rf))
print('prec dt:', metrics.precision_score(y_test, y_pred_dt))
print('prec rf:', metrics.precision_score(y_test, y_pred_rf))

In [ ]:
sars = pd.read_csv('../input/epitope-prediction/input_sars.csv')
sars

In [ ]:
protseqs = []

for i in sars.protein_seq:
    tam = len(i)
    protseqs.append(tam)

pepseqs = sars.end_position - sars.start_position

In [ ]:
protseqs = pd.DataFrame(protseqs)
sars['protein_seq']= (sars['protein_seq']==0).astype(int)
sars = sars.replace({'protein_seq': protseqs})

pepseqs = pd.DataFrame(pepseqs)
sars['peptide_seq'] = (sars['peptide_seq']==0).astype(int)
sars = sars.replace({'peptide_seq': pepseqs})

In [ ]:
alvo = sars[atributos]
y2_pred = rf.predict(alvo)
y = sars.target

lr_probs = rf.predict_proba(alvo)

lr_probs = lr_probs[:, 1]

lr_auc = metrics.roc_auc_score(y, lr_probs)

print('AUC RF: ROC AUC=%.3f' % (lr_auc))

TAREFA 2

In [ ]:
covid =  pd.read_csv('../input/epitope-prediction/input_covid.csv')
covid

In [ ]:
protseq = pd.DataFrame(protseq)
covid['protein_seq']= (covid['protein_seq']==0).astype(int)
covid = covid.replace({'protein_seq': protseq})

pepseq = pd.DataFrame(pepseq)
covid['peptide_seq'] = (covid['peptide_seq']==0).astype(int)
covid = covid.replace({'peptide_seq': pepseq})
covid

In [ ]:
covidf = covid[atributos]
covidf

In [ ]:
celb

In [ ]:
trfinal = celb.append(alvo)
trfinal

tefinal = bcell['target'].append(sars['target'])
tefinal

In [ ]:
#TREINO COM OS 2 DATASETS e TARGET FINAL (calcular separado e somar probabilidades)

treino_X = trfinal
teste_X = covidf
treino_y = tefinal

rfcov = RandomForestClassifier(n_estimators=30, random_state=1)
rfcov = rfcov.fit(treino_X, treino_y)
predcov = rf.predict(teste_X)

cov123 = pd.DataFrame(predcov)
cov123.hist(figsize=(20, 10), color = 'purple')

In [ ]:
final = pd.DataFrame(covidf)
final['target'] = predcov
final